In [2]:
import pandas as pd
import numpy as np

* NaN 用 0 取代
* 去除最後一筆資料 (結算最後比分)
* 加上隊伍資訊
* Action == OP 的 column 去除，reset index

In [52]:
def dataset_preprocess(df, team):
    df = df.fillna(0)
    df = df.drop(len(df)-1)
    df.insert(0, column='Team', value=[team for i in range(len(df))])
    df = df.drop(df[df['Action'] == 'OP'].index)
    df = df.reset_index(drop=True)
    return df

In [56]:
def get_data(link, team):
    df_1 = pd.read_excel(link, sheet_name = team + ' Set1')
    df_2 = pd.read_excel(link, sheet_name = team + ' Set2')
    df_3 = pd.read_excel(link, sheet_name = team + ' Set3')
    df_4 = pd.read_excel(link, sheet_name = team + ' Set4')
    df_5 = pd.read_excel(link, sheet_name = team + ' Set5')

    df_1 = dataset_preprocess(df_1, team)
    df_2 = dataset_preprocess(df_2, team)
    df_3 = dataset_preprocess(df_3, team)
    df_4 = dataset_preprocess(df_4, team)
    df_5 = dataset_preprocess(df_5, team)

    df = pd.concat([df_1, df_2, df_3, df_4, df_5])
    drop_col = df.columns.tolist()
    col = ['Team', 'No.', 'Space', 'Action', 'Erros', 'Attempts', 'Score']
    for c in col:
        drop_col.remove(c)
    df = df.drop(columns=drop_col)

    df = df.astype({'No.': 'int32'})
    df = df.astype({'No.': 'str'})

    df['Space'].replace({'M': 9, 'Y': 10, 'L': 11, 'FZ': 12}, inplace=True)
    df['Action'].replace({'A': 1, 'B': 2, 'D': 3, 'FS': 4, 'G': 5, 'JS': 6, 'OP': 7, 'R': 8}, inplace=True)

    return df

In [57]:
df_IRI = get_data('Data/2022 VNL/IRI VS POL.xlsx', 'IRI')
df_POL = get_data('Data/2022 VNL/IRI VS POL.xlsx', 'POL')
print(len(df_IRI), len(df_POL))

547 530


In [58]:
df_IRI.head()

,Team,No.,Space,Action,Erros,Attempts,Score
0,IRI,2,10,6,0.0,0.0,0.0
1,IRI,22,1,2,0.0,0.0,1.0
2,IRI,2,10,6,0.0,0.0,0.0
3,IRI,2,8,3,1.0,0.0,0.0
4,IRI,8,8,8,0.0,0.0,0.0


In [75]:
apprix_1 = df_IRI.iloc[:500,:]
apprix_2 = df_IRI.iloc[500:,:]

---
* Team、No. 做 one-hot encoding
* Space、Action 比照羽球 paper 辦理

In [10]:
space_col = ['Space']
action_col = ['Action']
other_col = [c for c in df_IRI_1.columns if(c not in space_col and c not in action_col)]

print(space_col)
print(action_col)
print(other_col)

['Space']
['Action']
['Team', 'No.', 'Erros', 'Attempts', 'Score']


In [11]:
# 分開並在訓練階段 space 和 action 單獨丟進自己的 embedding layer
space_encoding = df_IRI_1[space_col].values.astype('float32')
action_encoding = df_IRI_1[action_col].values.astype('float32')
other_encoding = df_IRI_1[other_col].values.astype('float32')

---
按照一球把data分開 (好像也無法)
